In [1]:
"""Run two below lines to get my_abc module"""
# !git clone https://github.com/thanhttttt/thanh.git
# !pip install -r /content/thanh/requirements.txt
"""Run two below lines to connect to drive"""
# from google.colab import drive
# drive.mount('/content/drive')

'Run two below lines to connect to drive'

# Import

In [2]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


import sys
sys.path.append('../')
sys.path.append('/content/thanh/')
sys.path.append('../thanh/')
import sionna

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.config.seed = 42 # Set seed for reproducible random number generation

# Load the required Sionna components
from sionna.nr.my_abc import *

# Load model weight

In [3]:
_model = CustomNeuralReceiver(training = False)
inputs = tf.zeros([1,48,14,18])
_model(inputs)
_model.summary()

#load_weights(_model, '/content/drive/MyDrive/Pusch_data/Model_weights/model_weight_FULL_RB_epoch_40.pkl')
# load_weights(_model, '../model_weight_FULL_RB_epoch_40.pkl')
load_weights(_model, '../weight_4RB_batchsize_1024_186k_sample_dynamic_config_epoch90.pkl')

Model: "custom_neural_receiver"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  20864     
                                                                 
 residual_block (ResidualBl  multiple                  639232    
 ock)                                                            
                                                                 
 residual_block_1 (Residual  multiple                  639232    
 Block)                                                          
                                                                 
 residual_block_2 (Residual  multiple                  639232    
 Block)                                                          
                                                                 
 residual_block_3 (Residual  multiple                  639232    
 Block)                                     

# Set up config

In [4]:
"""test setup"""
batch_size = 8

"""field config"""
# field_dir = '../data_field/dump_lan3_2dmrs'
field_dir = '../data_field/dump_lan2_2dmrs'

# field_name = 'dump_pass_sfn_696_sf_4'
field_name = 'dump_fail_sfn_525_sf_15'

"""channel setup"""
no = 2.
CDL_model = 'A'
delay_spread = 50
speed = 1

## create channel

In [5]:
channel_model = CDL(model = CDL_model,
                            delay_spread = delay_spread*1e-9,
                            carrier_frequency = CARRIER_FREQUENCY,
                            ut_array = Ue_Antenna,
                            bs_array = Gnb_AntennaArray,
                            direction = 'uplink',
                            min_speed = speed,
                            max_speed = speed)

## Set field config

In [6]:
cfg_path = f'{field_dir}/{field_name}_cfg.txt'
data_path = f'{field_dir}/{field_name}_freq.bin'

In [7]:
caseInfo, sysInfo, ueInfo, chcfg, auxInfo = config_parser(cfg_path)
freqIQ = data_reader(data_path, (sysInfo['NRxAnt'], 14, sysInfo['BwpNRb']*12))
ueIQ = tf.constant(freqIQ[None, None, :, :, ueInfo[0]['FirstPrb']*12:(ueInfo[0]['NPrb'] + ueInfo[0]['FirstPrb'])*12])

In [8]:
sysCfg = SystemConfig(**sysInfo)
ueCfg = UeConfig(**ueInfo[0])
myCfg = MyConfig(sysCfg, [ueCfg])
puschCfg = MyPUSCHConfig(myCfg, slot_number=caseInfo['sf'], frame_number=caseInfo['sfn'])
# puschCfg.show() # uncomment for detail

In [9]:
simulator = MySimulator(puschCfg)
channel = OFDMChannel(channel_model=channel_model, resource_grid=simulator.resource_grid,
                                    add_awgn=False, normalize_channel=True, return_channel=True)

In [10]:
b, c, y, x ,h = simulator.sim(batch_size, channel, no, return_tx_iq=True, return_channel=True)
r = simulator.ref(batch_size)

# Evaluate

## Eval on sim data

In [11]:
y.shape, r.shape

(TensorShape([8, 1, 8, 14, 1680]), TensorShape([8, 1, 1, 14, 1680]))

In [12]:
"""Evaluate"""
preds = predict(_model, y, r)
c_pred = tf.reshape(preds, [preds.shape[0], 1, c.shape[-1]])
b_hat, crc = simulator.TB_Decode(c_pred)
# loss_cal(c_pred, c)
compute_ber(b, b_hat), crc

I0000 00:00:1742351947.045696   29337 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


(<tf.Tensor: shape=(), dtype=float64, numpy=0.0>,
 <tf.Tensor: shape=(8, 1), dtype=bool, numpy=
 array([[ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True]])>)

In [13]:
h_est, x_hat, llr_det, b_hat, crc = simulator.rec(y)
compute_ber(b, b_hat), crc

(<tf.Tensor: shape=(), dtype=float64, numpy=0.11233786730945822>,
 <tf.Tensor: shape=(8, 1), dtype=bool, numpy=
 array([[False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False]])>)

## Eval on field data

In [28]:
h_est, x_hat, llr_det, b_hat, crc_1 = simulator.rec(ueIQ)
if crc_1:
    b_rec = b_hat
    c_rec = simulator.TB_Encoder(b_rec)
crc_1

<tf.Tensor: shape=(1, 1), dtype=bool, numpy=array([[ True]])>

In [29]:
np.sum(b_hat)

8672.0

In [35]:
llr_det[...,:12]

<tf.Tensor: shape=(1, 1, 1, 12), dtype=float32, numpy=
array([[[[-1328.7639, -1297.9176,  1398.7656,  1271.8197,  1569.591 ,
          -1551.1521,  1430.6047, -1410.9545, -1357.2728,  1682.3329,
           1497.721 ,  1358.4891]]]], dtype=float32)>

In [ ]:
# import scipy
# scipy.io.savemat('decode_vectors.mat', {'scbSoftBits':llr_det[0,0]})

In [32]:
"""Evaluate"""
preds = predict(_model, ueIQ, r[0:1])
c_pred = tf.reshape(preds, [preds.shape[0], 1, c.shape[-1]])
b_hat, crc_2 = simulator.TB_Decode(c_pred) 
if crc_1:
    print(loss_cal(c_pred, c_rec), compute_ber(b_hat, b_rec))
crc_2

tf.Tensor(0.2466464, shape=(), dtype=float32) tf.Tensor(0.014003673094582185, shape=(), dtype=float64)


<tf.Tensor: shape=(1, 1), dtype=bool, numpy=array([[False]])>